In [1]:
import json
import time
import random
import tqdm.auto as tqdm
from transformers import pipeline

In [2]:
with open('train_aug.jsonl', 'r') as f:
    train_data = f.readlines()

train_events = [json.loads(x) for x in train_data]
len(train_events)

5826

In [3]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')

In [11]:
fout = open('train_aug2.jsonl', 'w')

#### Transforming trigger_words using BERT MLM to generate alternate data samples

In [12]:
start_time = time.time()
result = []
outer_loop = tqdm.tqdm(train_events[:])
for item in outer_loop:
    result.append(item)
    fout.write(json.dumps(item)+'\n')
    new_item = json.loads(json.dumps(item))
    token_adj = 0
    for evt in new_item['events']:
        if evt['type'] in ['Causation', 'Process_start', 'Attack', 'Hostile_encounter', 'Catastrophe', 'Motion', 'Competition', 'Killing', 'Process_end', 'Social_event', 'Conquering', 'Statement', 'Self_motion', 'Arriving', 'Destroying', 'Coming_to_be', 'Bodily_harm', 'Death', 'Creating', 'Military_operation', 'Damaging', 'Cause_change_of_strength', 'Cause_change_of_position_on_a_scale', 'Hold', 'Control', 'Earnings_and_losses', 'Getting', 'Becoming', 'Arranging', 'Know', 'Preventing_or_letting', 'Presence', 'Escaping', 'Defending', 'Action', 'Motion_directional', 'Cause_to_be_included', 'Change', 'Traveling', 'Placing', 'Participation', 'Influence', 'Change_of_leadership', 'Judgment_communication', 'Expressing_publicly', 'Name_conferral', 'Request', 'Giving', 'Supporting', 'Recording']:
            # remove event types that occur frequently - we keep their number as-is in the dataset
            new_item['events'].remove(evt)
            #continue
        else:
            # double this range
            for mnt in evt['mention']:
        #        print(mnt)
                sent = new_item['content'][mnt['sent_id']]
                toks = sent['tokens'].copy()

        #        print(toks)
        #        print(len(toks))
                start = mnt['offset'][0]
                end = mnt['offset'][1]
                if start > len(toks) or end > len(toks):
                    continue
                orig_word = toks[start]
                del toks[start:end]
                toks.insert(start, '[MASK]')

                # the unmasker step takes 0.1 sec on average
                candidates = unmasker(' '.join(toks))

                out = random.choice(candidates[1:])

                toks[start] = out['token_str']
        #        print(len(toks))
        #        print(toks)
                sent['tokens'] = toks
                mnt['trigger_word'] = out['token_str']
                mnt['offset'] = [start, start+1]
        #        print(mnt)

    result.append(new_item)
    fout.write(json.dumps(new_item)+'\n')

fout.close()
len(result)

  0%|          | 0/5826 [00:00<?, ?it/s]

11652

In [10]:
fout.close()

In [ ]:
#idx=0
#print(result[idx]['events'][0],"\n")
#print(result[idx+1]['events'][0])

In [6]:
#train_events[799:800]

[{'title': 'Operation Grapple',
  'id': 'b0efcbf49faefcd8c265b7ee3db5ca9d',
  'content': [{'sentence': 'Operation Grapple was the name of four series of British nuclear weapons tests of early atomic bombs and hydrogen bombs carried out in 1957 and 1958 at Malden Island and Kiritimati (Christmas Island) in the Pacific Ocean as part of the British hydrogen bomb programme.',
    'tokens': ['Operation',
     'Grapple',
     'was',
     'the',
     'name',
     'of',
     'four',
     'series',
     'of',
     'British',
     'nuclear',
     'weapons',
     'tests',
     'of',
     'early',
     'atomic',
     'bombs',
     'and',
     'hydrogen',
     'bombs',
     'performed',
     'in',
     '1957',
     'and',
     '1958',
     'at',
     'Malden',
     'Island',
     'and',
     'Kiritimati',
     '(',
     'Christmas',
     'Island',
     ')',
     'in',
     'the',
     'Pacific',
     'Ocean',
     'as',
     'part',
     'of',
     'the',
     'British',
     'hydrogen',
     'bomb